In [3]:
import numpy as np
import pandas as pd
import json

In [4]:
exchange_info = pd.read_csv("../data/exchange/exchangeInfo.csv", index_col=[0])
exchange_info.head()

,status,baseAsset,baseAssetPrecision,quoteAsset,quotePrecision,quoteAssetPrecision,baseCommissionPrecision,quoteCommissionPrecision,orderTypes,icebergAllowed,ocoAllowed,quoteOrderQtyMarketAllowed,allowTrailingStop,isSpotTradingAllowed,isMarginTradingAllowed,filters,permissions
symbol,,,,,,,,,,,,,,,,,
ETHBTC,TRADING,ETH,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
LTCBTC,TRADING,LTC,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
BNBBTC,TRADING,BNB,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
NEOBTC,TRADING,NEO,8,BTC,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,True,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...","['SPOT', 'MARGIN']"
QTUMETH,TRADING,QTUM,8,ETH,8,8,8,8,"['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_...",True,True,True,True,True,False,"[{'filterType': 'PRICE_FILTER', 'minPrice': '0...",['SPOT']


In [5]:
available_assets = np.unique(np.append(exchange_info.baseAsset.values, exchange_info.quoteAsset.values))
len(available_assets)

506

In [7]:
json_data = {"nodes": [{"id": x, "group": 1} for x in available_assets],
            "links": [{"source": row["quoteAsset"], "target": row["baseAsset"], "value": 1} for i, row in exchange_info.iterrows()]}


In [8]:
with open('../crypto_market_network/crypto_market.json', 'w') as f:
    json.dump(json_data, f)

In [9]:
crypto_graph = {k: set() for k in available_assets}
for i, row in exchange_info.iterrows():
    crypto_graph[row['quoteAsset']].add(row['baseAsset'])

In [32]:
arbitrage_paths = {}
def find_arbitrage_paths(graph, start, current_path=[]):
    global arbitrage_paths
    if start not in arbitrage_paths:
        arbitrage_paths[start] = []
    
    current_node = graph[start] if len(current_path)==0 else graph[current_path[-1]]
    if len(current_node)==0:
        return None
    if start in current_node:
        arbitrage_paths[start] += [current_path + [start]]
        return None
    for future_node in current_node:
        if future_node not in current_path:
            find_arbitrage_paths(graph, start, current_path + [future_node])
    return None 

In [27]:
def find_all_paths(graph, start, end):
    paths = []
    def _find_all_paths(graph, start, end, current_path=[]):
        nonlocal paths
        current_node = graph[start] if len(current_path)==0 else graph[current_path[-1]]
        if len(current_node)==0:
            return None
        if end in current_node:
            paths += [current_path + [end]]
            return None
        for future_node in current_node:
            if future_node not in current_path:
                _find_all_paths(graph, future_node, end, current_path + [future_node])
        return None
    _find_all_paths(graph, start, end)
    paths = [[start] + x for x in paths]
    return paths

In [28]:
find_all_paths(crypto_graph, 'ETH', 'BTC')

[['ETH', 'PAX', 'BTC'],
 ['ETH', 'TUSD', 'BTC'],
 ['ETH', 'BNB', 'PAX', 'BTC'],
 ['ETH', 'BNB', 'TUSD', 'BTC'],
 ['ETH', 'BNB', 'DAI', 'BTC'],
 ['ETH', 'BNB', 'USDC', 'BTC']]

In [30]:
find_all_paths(crypto_graph, 'BTC', 'ETH')

[['BTC', 'ETH']]

In [33]:
find_arbitrage_paths(crypto_graph, 'BTC')

In [34]:
arbitrage_paths

{'BTC': [['ETH', 'PAX', 'BTC'],
  ['ETH', 'TUSD', 'BTC'],
  ['ETH', 'BNB', 'PAX', 'BTC'],
  ['ETH', 'BNB', 'TUSD', 'BTC'],
  ['ETH', 'BNB', 'DAI', 'BTC'],
  ['ETH', 'BNB', 'USDC', 'BTC'],
  ['UST', 'ETH', 'PAX', 'BTC'],
  ['UST', 'ETH', 'TUSD', 'BTC'],
  ['UST', 'ETH', 'BNB', 'PAX', 'BTC'],
  ['UST', 'ETH', 'BNB', 'TUSD', 'BTC'],
  ['UST', 'ETH', 'BNB', 'DAI', 'BTC'],
  ['UST', 'ETH', 'BNB', 'USDC', 'BTC'],
  ['UST', 'BNB', 'PAX', 'BTC'],
  ['UST', 'BNB', 'TUSD', 'BTC'],
  ['UST', 'BNB', 'DAI', 'BTC'],
  ['UST', 'BNB', 'USDC', 'BTC'],
  ['BNB', 'PAX', 'BTC'],
  ['BNB', 'TUSD', 'BTC'],
  ['BNB', 'DAI', 'BTC'],
  ['BNB', 'USDC', 'BTC'],
  ['TUSD', 'BTC'],
  ['PAX', 'BTC'],
  ['DAI', 'BTC']]}

In [8]:
for coin in available_assets:
    find_arbitrage_paths(crypto_graph, coin)
    arbitrage_paths[coin] = sorted(arbitrage_paths[coin], key=len)

In [11]:
for coin in available_assets:
    arbitrage_paths[coin] = sorted(arbitrage_paths[coin], key=len)

In [13]:
with open('../data/exchange/arbitrage_paths.json', 'w') as f:
    json.dump(arbitrage_paths, f)

In [12]:
arbitrage_paths['BTC']

[['TUSD', 'BTC'],
 ['PAX', 'BTC'],
 ['DAI', 'BTC'],
 ['ETH', 'TUSD', 'BTC'],
 ['ETH', 'PAX', 'BTC'],
 ['BNB', 'TUSD', 'BTC'],
 ['BNB', 'PAX', 'BTC'],
 ['BNB', 'DAI', 'BTC'],
 ['BNB', 'USDC', 'BTC'],
 ['UST', 'ETH', 'TUSD', 'BTC'],
 ['UST', 'ETH', 'PAX', 'BTC'],
 ['UST', 'BNB', 'TUSD', 'BTC'],
 ['UST', 'BNB', 'PAX', 'BTC'],
 ['UST', 'BNB', 'DAI', 'BTC'],
 ['UST', 'BNB', 'USDC', 'BTC'],
 ['ETH', 'BNB', 'TUSD', 'BTC'],
 ['ETH', 'BNB', 'PAX', 'BTC'],
 ['ETH', 'BNB', 'DAI', 'BTC'],
 ['ETH', 'BNB', 'USDC', 'BTC'],
 ['UST', 'ETH', 'BNB', 'TUSD', 'BTC'],
 ['UST', 'ETH', 'BNB', 'PAX', 'BTC'],
 ['UST', 'ETH', 'BNB', 'DAI', 'BTC'],
 ['UST', 'ETH', 'BNB', 'USDC', 'BTC']]